In [2]:
## MP4 HD Quality
'''
from yt_dlp import YoutubeDL

# YouTube Shorts video URL to download
video_url = 'https://www.youtube.com/shorts/p-La55XDF5k'

# Download settings
ydl_opts = {
    'format': 'bestvideo+bestaudio/best',  # Selects the best video and audio quality
    'outtmpl': 'youtube_shorts_video.%(ext)s',  # Output filename and extension
}

# Downloading the video
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

print("Video downloaded successfully!")


'''


##https://www.youtube.com/watch?v=ORMRfFYMwVU
'''
from yt_dlp import YoutubeDL

# YouTube Shorts video URL to download
video_url = 'https://www.youtube.com/shorts/p-La55XDF5k'

# Download settings (audio only)
ydl_opts = {
    'format': 'bestaudio/best',  # Selects the best audio quality
    'outtmpl': 'youtube_shorts_audio.%(ext)s',  # Output filename and extension
    'postprocessors': [{  # Converts the audio file to MP3 format
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',  # MP3 format
        'preferredquality': '192',  # Audio quality (192 kbps)
    }],
}

# Downloading the audio
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

print("Audio file downloaded successfully!")
'''

[youtube] Extracting URL: https://www.youtube.com/watch?v=ORMRfFYMwVU
[youtube] ORMRfFYMwVU: Downloading webpage
[youtube] ORMRfFYMwVU: Downloading ios player API JSON
[youtube] ORMRfFYMwVU: Downloading player e627e516


         n = E4NeEr2SKhnZpMa ; player = https://www.youtube.com/s/player/e627e516/player_ias.vflset/en_US/base.js
         n = bMwNpyXO2bCJrxc ; player = https://www.youtube.com/s/player/e627e516/player_ias.vflset/en_US/base.js


[youtube] ORMRfFYMwVU: Downloading m3u8 information
[info] ORMRfFYMwVU: Downloading 1 format(s): 140
[download] Destination: youtube_shorts_audio.m4a
[download] 100% of    1.75MiB in 00:00:00 at 2.94MiB/s   
[FixupM4a] Correcting container of "youtube_shorts_audio.m4a"
[ExtractAudio] Destination: youtube_shorts_audio.mp3
Deleting original file youtube_shorts_audio.m4a (pass -k to keep)
Ses dosyası başarıyla indirildi!


In [16]:
from PIL import Image

# Open the image
with Image.open("resim1.jpg") as img:
    # Get the original dimensions
    original_width, original_height = img.size
    
    # Set the new width
    new_width = 1080
    
    # Calculate the new height proportionally
    new_height = int((new_width / original_width) * original_height)
    
    # Resize the image
    resized_image = img.resize((new_width, new_height))
    
    # Save the new image
    resized_image.save("resized_image.jpg")

    print("Image resized successfully!")

Resim başarıyla yeniden boyutlandırıldı!


In [18]:
from moviepy.editor import VideoFileClip, AudioFileClip, ImageClip, clips_array

# Load the first video
video1 = VideoFileClip("video1.mp4").without_audio()  # Remove the audio

# The second content will be an image, setting its duration to match the first video
image = ImageClip("resized_image.jpg").set_duration(video1.duration)  # Convert the image to a video and set the same duration as the video

# Combine the two clips vertically (first the video, then the image)
combined_video = clips_array([[video1], [image]]).set_duration(video1.duration)  # Set the duration to match the first video

# Load the new audio file
new_audio = AudioFileClip("audio.mp3").subclip(0, video1.duration)  # Limit the audio duration to match the video

# Add the audio to the combined video
final_video = combined_video.set_audio(new_audio)

# Save the final video
final_video.write_videofile("final_video.mp4", codec="libx264", fps=24)

print("Final video created successfully!")


Moviepy - Building video final_combined_video.mp4.
MoviePy - Writing audio in final_combined_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video final_combined_video.mp4



Moviepy - Done !
Moviepy - video ready final_combined_video.mp4


In [1]:
import os
import pickle
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from google.auth.transport.requests import Request
from moviepy.editor import VideoFileClip, AudioFileClip, ImageClip, clips_array
from googleapiclient.http import MediaFileUpload
from googletrans import Translator
from datetime import datetime, timedelta
from yt_dlp import YoutubeDL  # YouTubeDL module
import random
import re  # For extracting hashtags

# Google API credentials and YouTube Data API service configuration
scopes = ["https://www.googleapis.com/auth/youtube.upload", "https://www.googleapis.com/auth/youtube.force-ssl"]

# Random title and description generator
adjectives = ["Hilarious", "Epic", "Crazy", "Amazing", "Funny", "Incredible"]
verbs = ["Moments", "Challenges", "Pranks", "Skits", "Reactions", "Fails"]
topics = ["Friends", "Daily Life", "Adventures", "Comedy Show", "Vlogs", "Entertainment"]

def generate_title():
    adj = random.choice(adjectives)
    verb = random.choice(verbs)
    topic = random.choice(topics)
    return f"{adj} {verb} with {topic} - Must Watch!"

def generate_description():
    return f"Enjoy these {random.choice(adjectives)} {random.choice(verbs)} from {random.choice(topics)}! Don't forget to like and subscribe for more."

def get_authenticated_service():
    credentials = None
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            credentials = pickle.load(token)

    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
                "client_secret.json", scopes)
            credentials = flow.run_local_server(port=0)

        with open("token.pickle", "wb") as token:
            pickle.dump(credentials, token)

    return googleapiclient.discovery.build("youtube", "v3", credentials=credentials)

def upload_video(youtube, video_file, title, description, tags, category_id, privacy_status, publish_at=None, video_language="en"):
    body = {
        "snippet": {
            "title": title,
            "description": description,
            "tags": tags + ["#shorts"],  # Adding Shorts tag
            "categoryId": category_id,
            "defaultLanguage": "en",  # Video title and description language
            "defaultAudioLanguage": video_language  # Spoken language of the video
        },
        "status": {
            "privacyStatus": privacy_status,
            "selfDeclaredMadeForKids": False  # Not marked for kids
        }
    }

    if publish_at:
        body["status"]["publishAt"] = publish_at

    media_file = MediaFileUpload(video_file, resumable=True)

    request = youtube.videos().insert(
        part="snippet,status",
        body=body,
        media_body=media_file
    )

    response = None
    while response is None:
        try:
            status, response = request.next_chunk()
            if status:
                print(f"Upload Status: {int(status.progress() * 100)}% completed.")
        except googleapiclient.errors.HttpError as e:
            if e.resp.status in [500, 502, 503, 504]:
                print("Server error, retrying...")
            elif e.resp.status == 403:
                print("Quota limit exceeded. Please increase your quota or try again later.")
                break
            else:
                raise

    if response:
        print("Video uploaded successfully!")
        print(f"https://www.youtube.com/watch?v={response['id']}")
        return response['id']
    return None

def translate_text(text, dest_lang='tr'):
    """Translates the given text to the specified language (default: Turkish)"""
    translator = Translator()
    try:
        translated = translator.translate(text, dest=dest_lang)
        if translated.text:
            return translated.text
        else:
            print(f"Translation returned empty: {text} -> {dest_lang}")
            return text  # Return original text if translation is empty
    except Exception as e:
        print(f"An error occurred: {e}")
        return text  # Return original text if there's an error

def add_multiple_languages_to_video(youtube, video_id, translations):
    """Adds multiple languages to a YouTube video"""
    body = {
        "id": video_id,
        "localizations": translations
    }

    request = youtube.videos().update(
        part="localizations",
        body=body
    )

    response = request.execute()
    print("All languages added successfully!")
    return response

def get_top_30_languages_translations(title, description):
    """Translates the title and description into the top 30 most spoken languages"""
    languages = ['tr', 'fr', 'it', 'es', 'de', 'zh-cn', 'hi', 'ar', 'ru', 'pt', 
                 'ja', 'ko', 'bn', 'vi', 'ur', 'pa', 'fa', 'id', 'th', 'ms', 
                 'pl', 'ro', 'uk', 'nl', 'el', 'sv', 'hu', 'cs', 'da', 'fi']
    
    translations = {}
    for lang in languages:
        translated_title = translate_text(title, dest_lang=lang)
        translated_description = translate_text(description, dest_lang=lang)
        translations[lang] = {
            "title": translated_title,
            "description": translated_description
        }
    return translations

def get_video_hashtags(description):
    # Using a simple regex to identify hashtags
    hashtags = re.findall(r"#\w+", description)
    return hashtags

def get_video_details(youtube, video_id):
    # Get video details including description
    request = youtube.videos().list(
        part="snippet",
        id=video_id
    )
    response = request.execute()
    return response['items']

if __name__ == "__main__":
    youtube = get_authenticated_service()

    # Get hashtags from a popular video using its video_id
    popular_video_id = "TTMZpCcuaZ8"  # Enter the ID of a popular video here
    popular_video_details = get_video_details(youtube, popular_video_id)
    
    # Extract hashtags from the description of the popular video
    popular_video_description = popular_video_details[0]['snippet']['description']
    popular_hashtags = get_video_hashtags(popular_video_description)
    
    # Process the first video before uploading
    video1 = VideoFileClip("video1.mp4").without_audio()  # Remove audio
    image = ImageClip("resized_image.jpg").set_duration(video1.duration)  # Convert the image to a video and set the same duration as the video
    combined_video = clips_array([[video1], [image]]).set_duration(video1.duration)  # Set the duration to match the first video
    new_audio = AudioFileClip("audio.mp3").subclip(0, video1.duration)  # Limit the audio duration to match the video
    final_video = combined_video.set_audio(new_audio)
    final_video.write_videofile("final_video.mp4", codec="libx264", fps=24)  # Save the final video

    # Generate random title and description
    title = generate_title()
    description = generate_description()
    
    # Fixed tags
    manual_tags = ["fyp", "shorts", "viral", "Entertainment", "Comedy", "Humor", "DailyLife", "Vlog", "Sketch", "Trending", "FunnyMoments", "FunVideos", "Challenge", "Parody", "Prank", "Viral", "FunnyVideos", "Laugh"]
    
    # Combine manual hashtags with those from the popular video
    combined_tags = manual_tags + popular_hashtags
    
    category_id = "23"  # YouTube category, 23 = Entertainment
    privacy_status = "public"  # Privacy status of the video: "public", "private", or "unlisted"

    # Upload the video to YouTube and get the video ID
    video_id = upload_video(youtube, "final_video.mp4", title, description, combined_tags, category_id, privacy_status)
    
    if video_id:
        # Get translations for the top 30 spoken languages
        translations = get_top_30_languages_translations(title, description)
        
        # Add multiple languages to the video
        add_multiple_languages_to_video(youtube, video_id, translations)


Moviepy - Building video final_video.mp4.
MoviePy - Writing audio in final_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video final_video.mp4



Moviepy - Done !
Moviepy - video ready final_video.mp4
Video başarıyla yüklendi!
https://www.youtube.com/watch?v=mAexTeMuINk
Tüm diller başarıyla eklendi!


In [3]:
def get_video_categories(youtube):
    request = youtube.videoCategories().list(
        part="snippet",
        regionCode="US"  # Country Code (Examp, Italy "IT")
    )
    response = request.execute()
    
    for item in response["items"]:
        print(f"Category ID: {item['id']}, Title: {item['snippet']['title']}")

if __name__ == "__main__":
    youtube = get_authenticated_service()
    get_video_categories(youtube)

Category ID: 1, Title: Film & Animation
Category ID: 2, Title: Autos & Vehicles
Category ID: 10, Title: Music
Category ID: 15, Title: Pets & Animals
Category ID: 17, Title: Sports
Category ID: 18, Title: Short Movies
Category ID: 19, Title: Travel & Events
Category ID: 20, Title: Gaming
Category ID: 21, Title: Videoblogging
Category ID: 22, Title: People & Blogs
Category ID: 23, Title: Comedy
Category ID: 24, Title: Entertainment
Category ID: 25, Title: News & Politics
Category ID: 26, Title: Howto & Style
Category ID: 27, Title: Education
Category ID: 28, Title: Science & Technology
Category ID: 29, Title: Nonprofits & Activism
Category ID: 30, Title: Movies
Category ID: 31, Title: Anime/Animation
Category ID: 32, Title: Action/Adventure
Category ID: 33, Title: Classics
Category ID: 34, Title: Comedy
Category ID: 35, Title: Documentary
Category ID: 36, Title: Drama
Category ID: 37, Title: Family
Category ID: 38, Title: Foreign
Category ID: 39, Title: Horror
Category ID: 40, Title: Sci

In [1]:
import os
import pickle
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from google.auth.transport.requests import Request
from moviepy.editor import VideoFileClip, AudioFileClip, ImageClip, clips_array
from googleapiclient.http import MediaFileUpload
from googletrans import Translator
from datetime import datetime, timedelta
from yt_dlp import YoutubeDL  # YouTubeDL module
import isodate  # For handling durations in ISO 8601 format
import warnings
import random
import re  # For extracting hashtags

warnings.filterwarnings("ignore")
# Hide warnings

# Google API credentials and YouTube Data API service configuration
scopes = ["https://www.googleapis.com/auth/youtube.upload", "https://www.googleapis.com/auth/youtube.force-ssl", "https://www.googleapis.com/auth/youtube.readonly"]

# Random title and description generator
adjectives = ["Hilarious", "Epic", "Crazy", "Amazing", "Funny", "Incredible"]
verbs = ["Moments", "Challenges", "Pranks", "Skits", "Reactions", "Fails"]
topics = ["Friends", "Daily Life", "Adventures", "Comedy Show", "Vlogs", "Entertainment"]

def generate_title():
    adj = random.choice(adjectives)
    verb = random.choice(verbs)
    topic = random.choice(topics)
    return f"{adj} {verb} with {topic} - Must Watch!"

def generate_description():
    return f"Enjoy these {random.choice(adjectives)} {random.choice(verbs)} from {random.choice(topics)}! Don't forget to like and subscribe for more."

def get_authenticated_service():
    credentials = None
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            credentials = pickle.load(token)

    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            try:
                credentials.refresh(Request())
            except google.auth.exceptions.RefreshError:
                print("Token refresh error. Starting a new authentication process.")
                os.remove("token.pickle")  # Delete old token if there is an error
                flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
                    "client_secret.json", scopes)
                credentials = flow.run_local_server(port=0)
        else:
            flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
                "client_secret.json", scopes)
            credentials = flow.run_local_server(port=0)

        with open("token.pickle", "wb") as token:
            pickle.dump(credentials, token)

    return googleapiclient.discovery.build("youtube", "v3", credentials=credentials)

def upload_video(youtube, video_file, title, description, tags, category_id, privacy_status, publish_at=None, video_language="en"):
    body = {
        "snippet": {
            "title": title,
            "description": description,
            "tags": tags + ["#shorts"],  # Adding Shorts tag
            "categoryId": category_id,
            "defaultLanguage": "en",  # Video title and description language
            "defaultAudioLanguage": video_language  # Spoken language of the video
        },
        "status": {
            "privacyStatus": privacy_status,
            "selfDeclaredMadeForKids": False  # Set as not made for kids
        }
    }

    if publish_at:
        body["status"]["publishAt"] = publish_at

    media_file = MediaFileUpload(video_file, resumable=True)

    request = youtube.videos().insert(
        part="snippet,status",
        body=body,
        media_body=media_file
    )

    response = None
    while response is None:
        try:
            status, response = request.next_chunk()
            if status:
                print(f"Upload Status: {int(status.progress() * 100)}% completed.")
        except googleapiclient.errors.HttpError as e:
            if e.resp.status in [500, 502, 503, 504]:
                print("Server error, retrying...")
            elif e.resp.status == 403:
                print("Quota limit exceeded. Please increase your quota or try again later.")
                break
            else:
                raise

    if response:
        print("Video uploaded successfully!")
        print(f"https://www.youtube.com/watch?v={response['id']}")
        return response['id']
    return None

def translate_text(text, dest_lang='tr'):
    """Translates the given text to the specified language (default: Turkish)"""
    translator = Translator()
    try:
        translated = translator.translate(text, dest=dest_lang)
        if translated.text:
            return translated.text
        else:
            print(f"Translation returned empty: {text} -> {dest_lang}")
            return text  # Return original text if translation is empty
    except Exception as e:
        print(f"An error occurred: {e}")
        return text  # Return original text if there's an error

def add_multiple_languages_to_video(youtube, video_id, translations):
    """Adds multiple languages to a YouTube video"""
    body = {
        "id": video_id,
        "localizations": translations
    }

    request = youtube.videos().update(
        part="localizations",
        body=body
    )

    response = request.execute()
    print("All languages added successfully!")
    return response

def get_top_30_languages_translations(title, description):
    """Translates the title and description into the top 30 most spoken languages"""
    languages = ['tr', 'fr', 'it', 'es', 'de', 'zh-cn', 'hi', 'ar', 'ru', 'pt', 
                 'ja', 'ko', 'bn', 'vi', 'ur', 'pa', 'fa', 'id', 'th', 'ms', 
                 'pl', 'ro', 'uk', 'nl', 'el', 'sv', 'hu', 'cs', 'da', 'fi']
    
    translations = {}
    for lang in languages:
        translated_title = translate_text(title, dest_lang=lang)
        translated_description = translate_text(description, dest_lang=lang)
        translations[lang] = {
            "title": translated_title,
            "description": translated_description
        }
    return translations

def get_video_hashtags(description):
    hashtags = re.findall(r"#\w+", description)
    return hashtags

def get_video_details(youtube, video_ids):
    # Retrieve video details including language and duration
    request = youtube.videos().list(
        part="snippet,contentDetails",  # contentDetails to get duration
        id=",".join(video_ids)
    )
    response = request.execute()
    return response['items']

def is_video_short_enough(video):
    duration = video['contentDetails']['duration']
    duration_seconds = parse_duration_to_seconds(duration)
    return duration_seconds <= 30

def parse_duration_to_seconds(duration):
    return isodate.parse_duration(duration).total_seconds()

def is_video_in_english(video):
    language = video['snippet'].get('defaultAudioLanguage', video['snippet'].get('defaultLanguage', ''))
    return language == 'en'

def get_trending_entertainment_shorts_videos_in_usa(youtube, max_results=10):
    one_week_ago = datetime.utcnow() - timedelta(days=1)
    published_after = one_week_ago.isoformat("T") + "Z"
    
    request = youtube.search().list(
        part="snippet",
        type="video",
        order="viewCount",  
        videoDuration="short",  
        publishedAfter=published_after,
        regionCode="US",  
        relevanceLanguage="en",  
        videoCategoryId="10",  
        maxResults=max_results * 2  
    )
    response = request.execute()

    video_ids = [item['id']['videoId'] for item in response['items']]
    
    video_details = get_video_details(youtube, video_ids)
    
    short_english_videos = [video for video in video_details if is_video_short_enough(video) and is_video_in_english(video)]

    return short_english_videos[:max_results]

def download_video(video_id, output_filename="video1.mp4"):
    if os.path.exists(output_filename):
        os.remove(output_filename)
        print(f"Previous {output_filename} file deleted.")

    video_url = f"https://www.youtube.com/watch?v={video_id}"
    
    ydl_opts = {
        'format': 'bestvideo+bestaudio/best', 
        'outtmpl': output_filename,  
    }
    
    with YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])
    print(f"Video downloaded and saved as {output_filename}!")

if __name__ == "__main__":
    youtube = get_authenticated_service()

    # Get trending English short videos in the USA
    trending_videos = get_trending_entertainment_shorts_videos_in_usa(youtube, max_results=10)

    if trending_videos:
        # Download the most popular video
        video_id = trending_videos[0]['id']
        download_video(video_id, output_filename="video1.mp4")
        
        # Process the video
        video1 = VideoFileClip("video1.mp4").without_audio()  
        image = ImageClip("resized_image.jpg").set_duration(video1.duration)  
        combined_video = clips_array([[video1], [image]]).set_duration(video1.duration)  
        new_audio = AudioFileClip("audio.mp3").subclip(0, video1.duration)  
        final_video = combined_video.set_audio(new_audio)
        final_video.write_videofile("final_video.mp4", codec="libx264", fps=24)  

        # Generate random title and description
        title = generate_title()
        description = generate_description()

        # Fixed tags
        manual_tags = ["fyp", "shorts", "viral", "Entertainment", "Comedy", "Humor", "DailyLife", "Vlog", "Sketch", "Trending", "FunnyMoments", "FunVideos", "Challenge", "Parody", "Prank", "Viral", "FunnyVideos", "Laugh"]

        # Combine manual hashtags with hashtags from the popular video
        popular_hashtags = get_video_hashtags(trending_videos[0]['snippet']['description'])
        combined_tags = manual_tags + popular_hashtags

        category_id = "23"  
        privacy_status = "public"  

        # Upload the video to YouTube
        video_id = upload_video(youtube, "final_video.mp4", title, description, combined_tags, category_id, privacy_status)

        if video_id:
            translations = get_top_30_languages_translations(title, description)
            add_multiple_languages_to_video(youtube, video_id, translations)


Önceki video1.mp4 dosyası silindi.
[youtube] Extracting URL: https://www.youtube.com/watch?v=rYDI0Vgh_3Q
[youtube] rYDI0Vgh_3Q: Downloading webpage
[youtube] rYDI0Vgh_3Q: Downloading ios player API JSON
[youtube] rYDI0Vgh_3Q: Downloading player a62d836d


         n = tpNgldwfiHhvRTvBQ ; player = https://www.youtube.com/s/player/a62d836d/player_ias.vflset/en_US/base.js
         n = jEBnughRg5XBlARLm ; player = https://www.youtube.com/s/player/a62d836d/player_ias.vflset/en_US/base.js


[youtube] rYDI0Vgh_3Q: Downloading m3u8 information
[info] Testing format 616
[info] rYDI0Vgh_3Q: Downloading 1 format(s): 616+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 1
[download] Destination: video1.f616.mp4
[download] 100% of    2.95MiB in 00:00:00 at 4.42MiB/s                 
[download] Destination: video1.f140.m4a
[download] 100% of   92.59KiB in 00:00:00 at 411.68KiB/s 
[Merger] Merging formats into "video1.mp4"
Deleting original file video1.f616.mp4 (pass -k to keep)
Deleting original file video1.f140.m4a (pass -k to keep)
Video başarıyla indirildi ve video1.mp4 olarak kaydedildi!
Moviepy - Building video final_video.mp4.
MoviePy - Writing audio in final_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video final_video.mp4



Moviepy - Done !
Moviepy - video ready final_video.mp4
Video başarıyla yüklendi!
https://www.youtube.com/watch?v=4nrVjTtFE2A
Tüm diller başarıyla eklendi!
